In [1]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
from skimage import color
import skimage.io as io
import skimage.transform as trans
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from PIL import Image
from _shared_code import delete_images
import keras


def adjustData(img, mask, flag_multi_class, num_class):
    # if flag_multi_class:
    #     img = img / 255.0
    #     mask = mask[:, :, :, 0] if (len(mask.shape) == 4) else mask[:, :, 0]
    #     new_mask = np.zeros(mask.shape + (num_class,))
    #     for i in range(num_class):
    #         # for one pixel in the image, find the class in mask and convert it into one-hot vector
    #         # index = np.where(mask == i)
    #         # index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
    #         # new_mask[index_mask] = 1
    #         new_mask[mask == i, i] = 1
    #     new_mask = np.reshape(new_mask, (new_mask.shape[0], new_mask.shape[1] * new_mask.shape[2], new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
    #     mask = new_mask
    if (np.max(img) > 1):
        img = img / 255.0
    if (np.max(mask) > 1):
        mask = mask / 255.0
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return (img, mask)


def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode="grayscale",
                   mask_color_mode="grayscale", image_save_prefix="image", mask_save_prefix="mask",
                   flag_multi_class=False, num_class=2, save_to_dir=None, target_size=(256, 256), seed=1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed)
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        img, mask = adjustData(img, mask, flag_multi_class, num_class)
        yield (img, mask)


def validationGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode="grayscale",
                   mask_color_mode="grayscale", image_save_prefix="image", mask_save_prefix="mask",
                   flag_multi_class=False, num_class=2, save_to_dir=None, target_size=(256, 256), seed=1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed)
    train_generator = zip(image_generator, mask_generator)
    for (img, mask) in train_generator:
        img, mask = adjustData(img, mask, flag_multi_class, num_class)
        yield (img, mask)


def unet(pretrained_weights=None, input_size=(256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(learning_rate = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
    # model.compile(optimizer=Adam(learning_rate=.00001), loss='binary_crossentropy', metrics=['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model




2024-03-15 16:09:18.728787: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 16:09:18.766391: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 16:09:18.766430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 16:09:18.767610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 16:09:18.774259: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# base_folder = '/home/shannon/local/Source/Python/unet/data/membrane'
base_folder = '/home/shannon/local/Source/Python/bm_study/run_03'
source_images = base_folder + '/train'

# Create the augmented images directory if it doesn't exist
augmented_images = os.path.join(base_folder + '/train', 'aug')
os.makedirs(augmented_images, exist_ok=True)
delete_images(augmented_images)



In [3]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    

2024-03-15 16:09:20.904386: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
from keras import utils
data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')
myGene = trainGenerator(batch_size=2, train_path=source_images, image_folder='image', mask_folder='label', aug_dict=data_gen_args, save_to_dir=augmented_images, seed=10)
# myGenevalid = validationGenerator(batch_size=2, train_path=source_images, image_folder='image', mask_folder='label', aug_dict={})
model = unet()
model_checkpoint = ModelCheckpoint('unet_bm.keras', monitor='loss', verbose=1, save_best_only=True)
utils.set_random_seed(10)
model.fit(myGene, steps_per_epoch=2000, epochs=30, callbacks=[model_checkpoint])


Found 27 images belonging to 1 classes.
Found 27 images belonging to 1 classes.
Epoch 1/30
  58/2000 [..............................] - ETA: 10:43 - loss: 0.6708 - accuracy: 0.7133

KeyboardInterrupt: 

In [ ]:
# Sky = [128,128,128]
# Building = [128,0,0]
# Pole = [192,192,128]
# Road = [128,64,128]
# Pavement = [60,40,222]
# Tree = [128,128,0]
# SignSymbol = [192,128,128]
# Fence = [64,64,128]
# Car = [64,0,128]
# Pedestrian = [64,64,0]
# Bicyclist = [0,128,192]
# Unlabelled = [0,0,0]

# COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement, Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

# def labelVisualize(num_class, color_dict, img):
#     img = img[:, :, 0] if len(img.shape) == 3 else img
#     img_out = np.zeros(img.shape + (3,))
#     for i in range(num_class):
#         img_out[img == i, :] = color_dict[i]
#     return img_out / 255.0


def saveResult(file_list, npyfile, flag_multi_class=False, num_class=2):
    for i, item in enumerate(npyfile):
        # item[item < 0.2] = 0        # Added by me
        # item[item >= 0.2] = 1       # Added by me
        # img = labelVisualize(num_class, COLOR_DICT, item) if flag_multi_class else item[:, :, 0]
        img = item[:, :, 0]
        img = (img * 255).astype(np.uint8)  # Convert to uint8
        img = Image.fromarray(img, mode='L')  # Convert to grayscale image
        filename = os.path.splitext(file_list[i])[0]
        img.save(filename + "_predict.png")


def testGenerator(file_list, num_image=30, target_size=(256, 256), flag_multi_class=False, as_gray=True):
    for filename in file_list:
        img = io.imread(filename, as_gray=as_gray)
        img = img / 255.0
        img = trans.resize(img, target_size)
        img = np.reshape(img, img.shape + (1,)) if (not flag_multi_class) else img
        img = np.reshape(img, (1,) + img.shape)
        yield img


source_folder = "run_03/test"
file_list = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.endswith('.png') and 'actual' not in f and 'predict' not in f]
testGene = testGenerator(file_list)
results = model.predict(testGene, len(file_list), verbose=1)
saveResult(file_list, results)


In [ ]:
import os
from sklearn.metrics import accuracy_score
# loop through the folder images/originals/resized and load each image
# for each image, predict the mask and save it to images/predictions
# for each image, compare the predicted mask with the actual mask and calculate the accuracy
# save the accuracy to a file

folder_path = "images/originals/resized"
image_files = os.listdir(folder_path)

for file_name in image_files:
    file_path = os.path.join(folder_path, file_name)
    # Load the image using PIL
    img = Image.open(file_path)
    img = np.array(img)

    # run model.predict on the image
    results = model.predict(img, verbose=1)

    # load the mask for file_name from the folder images/masks/flood_filled
    mask_path = "images/masks/flood_filled"
    mask_file = os.path.join(mask_path, file_name)
    mask = Image.open(mask_file)
    mask = np.array(mask)

    # compare the predicted mask with the actual mask and calculate the accuracy
    accuracy = accuracy_score(mask.flatten(), results.flatten())

    # create a new filename by adding accuracy as a two digit number to the start of the original filename
    new_file_name = "{:02d}_{}".format(int(accuracy * 100), file_name)

    # save this file in a folder called ranked_predictions
    ranked_predictions = "run03/predictions_ranked"
    new_file_path = os.path.join(ranked_predictions, new_file_name)
    Image.fromarray(results).save(new_file_path)

    
